In [22]:
#Goal: create an app that accepts a stock ticker input from the user and plots closing price data for one month 

#Libraries
import pandas as pd
from bokeh.io import output_notebook #only for development. delete in production
from bokeh.plotting import figure, output_file, save, show
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter, DatetimeTicker

In [3]:
#import .env file to protect API Key during development
from dotenv import dotenv_values
config = dotenv_values(".env")

api_key = config['API_KEY']

In [ ]:
#Build in some interactivity by having the user submit a form which determines which data is requested.

#variables needed to choose from:
    #year_month = datetime index from 
    #symbol

In [4]:
#get the data from the AlphaVantage API
import requests
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&outputsize=full&apikey={api_key}'
r = requests.get(url)
data = r.json()

In [5]:
#Create a pandas dataframe with the data.
df = pd.DataFrame(data['Time Series (Daily)']).T.reset_index()

#set up the index as datetimes
df['index'] = pd.to_datetime(df['index'], infer_datetime_format = True) 
df.set_index('index', drop = True, inplace = True)
#rename columns
df.columns = ['open','high','low','close','adjusted_close','volume','dividend_amt','split_coeff']

df.index

DatetimeIndex(['2021-09-14', '2021-09-13', '2021-09-10', '2021-09-09',
               '2021-09-08', '2021-09-07', '2021-09-03', '2021-09-02',
               '2021-09-01', '2021-08-31',
               ...
               '1999-11-12', '1999-11-11', '1999-11-10', '1999-11-09',
               '1999-11-08', '1999-11-05', '1999-11-04', '1999-11-03',
               '1999-11-02', '1999-11-01'],
              dtype='datetime64[ns]', name='index', length=5503, freq=None)

In [6]:
#Bokeh integration w/ Jupyter notebook. Not needed in final production
output_notebook()

Loading BokehJS ...

In [28]:
#Create a Bokeh plot from the dataframe.

#reduce data to a month
month = df.loc['1999-11']

#some variables for plotting labels
months_list = ['January','February','March','April','May','June','July','August','September','October','November','December']
ticker_symbol = data['Meta Data']['2. Symbol']


#build the figure
p = figure(x_axis_type="datetime", sizing_mode="stretch_both",
           title=f"{ticker_symbol} Stock Prices - {months_list[month.index.month[-1]-1]} {month.index.year[-1]}")
p.xgrid.grid_line_color=None
p.ygrid.grid_line_alpha=0.5

p.xaxis.axis_label = 'Date'
p.xaxis.formatter = DatetimeTickFormatter(days="%D")
p.xaxis.ticker = DatetimeTicker(desired_num_ticks = len(month))

p.yaxis.axis_label = 'Price (per share)'
p.yaxis[0].formatter = NumeralTickFormatter(format="$0.00")


#add renderers
p.line(month.index, month.adjusted_close, legend_label="Closing Price (Adjusted)", line_color="purple", line_width=2)

show(p)